# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_df = pd.read_csv("../WeatherPy/weather_data.csv")
data_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Punta Arenas,-53.1500,-70.9167,51.91,50,40,42.57,CL,1662743287
1,1,Goderich,43.7501,-81.7165,74.91,56,0,9.04,CA,1662743288
2,2,Sumbe,-11.2061,13.8437,72.37,84,33,6.58,AO,1662743290
3,3,Chapais,49.7834,-74.8492,77.67,38,71,12.50,CA,1662743292
4,4,Sitka,57.0531,-135.3300,57.09,82,100,4.61,US,1662743293


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [82]:
gmaps.configure(api_key = g_key)
fig = gmaps.figure()
locations = data_df[["Latitude", "Longitude"]]
weights = data_df["Humidity"]
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, max_intensity=max(data_df["Humidity"])))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [84]:
clean_data_df = data_df
# A max temperature lower than 80 degrees but higher than 70
clean_data_df = clean_data_df.loc[(clean_data_df["Max Temperature"]<80) & (clean_data_df["Max Temperature"]>70)]
# Wind speed less than 10 mph.
clean_data_df = clean_data_df.loc[(clean_data_df["Wind Speed"]<10)]
# Zero cloudiness.
clean_data_df = clean_data_df.loc[(clean_data_df["Cloudiness"]==0)]

clean_data_df = clean_data_df.dropna()
clean_data_df = clean_data_df.drop(["Unnamed: 0"], axis = 1)
clean_data_df = clean_data_df.reset_index(drop=True)
clean_data_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Goderich,43.7501,-81.7165,74.91,56,0,9.04,CA,1662743288
1,Pombia,45.6530,8.6336,79.86,56,0,4.61,IT,1662743372
2,Albany,42.6001,-73.9662,79.66,63,0,3.78,US,1662743422
3,Erzin,36.9559,36.2000,78.57,42,0,3.60,TR,1662743435
4,Al Bardīyah,31.7561,25.0865,78.04,56,0,9.01,LY,1662743563
5,Broome,42.2506,-75.8330,79.66,58,0,5.75,US,1662743653
6,Mastic Beach,40.7668,-72.8521,79.11,67,0,7.00,US,1662743465
7,Pacific Grove,36.6177,-121.9166,75.42,87,0,8.99,US,1662743712
8,Dekernes,31.0876,31.5950,76.26,70,0,7.49,EG,1662743778
9,Diego de Almagro,-26.3667,-70.0500,71.67,11,0,8.52,CL,1662743804


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [85]:
hotel_df = clean_data_df
locations = {}
hotel_df
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    locations = f"{lat},{lng}"
    full_url = f"{url}{locations}&radius=5000&types=lodging&key={g_key}"
    response = requests.get(full_url).json()
    results = response["results"]
    
    try:
        print(results[0]["name"])
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except:
        print("missing result")
    

Harmony Inn
Albergo Fontana
missing result
Artemis Otel
بورتو بردية
Chenango Valley State Park
Brushwood House by Smith Point Beach
Lovers Point Inn
زكريا عبد الفتاح سليم
hotel luna del desierto
Beau Site
missing result
Nsanje Discovery Lodge
Sultan Hotel
Ditlhapi Guest House


In [86]:
hotel_df = hotel_df.dropna()
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Goderich,43.7501,-81.7165,74.91,56,0,9.04,CA,1662743288,Harmony Inn
1,Pombia,45.6530,8.6336,79.86,56,0,4.61,IT,1662743372,Albergo Fontana
2,Erzin,36.9559,36.2000,78.57,42,0,3.60,TR,1662743435,Artemis Otel
3,Al Bardīyah,31.7561,25.0865,78.04,56,0,9.01,LY,1662743563,بورتو بردية
4,Broome,42.2506,-75.8330,79.66,58,0,5.75,US,1662743653,Chenango Valley State Park
5,Mastic Beach,40.7668,-72.8521,79.11,67,0,7.00,US,1662743465,Brushwood House by Smith Point Beach
6,Pacific Grove,36.6177,-121.9166,75.42,87,0,8.99,US,1662743712,Lovers Point Inn
7,Dekernes,31.0876,31.5950,76.26,70,0,7.49,EG,1662743778,زكريا عبد الفتاح سليم
8,Diego de Almagro,-26.3667,-70.0500,71.67,11,0,8.52,CL,1662743804,hotel luna del desierto
9,Marsá Maţrūḩ,31.3525,27.2453,78.60,58,0,9.22,EG,1662744048,Beau Site


In [90]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [91]:
# Add marker layer ontop of heat map
    markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))